In [5]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.7.0
tiktoken version: 0.9.0


In [ ]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = {"https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt"}
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)
    

In [10]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("raw_text length:", len(raw_text))
print("raw_text sample:", raw_text[:100])  # Print the first 1000 characters

raw_text length: 20479
raw_text sample: I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [ ]:
import re
text = raw_text[:100]
result = re.split(r'(,.|\s+)', text)  # Replace multiple spaces with a single space
print(result)

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius--though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough--so', ' ', 'it', ' ', 'was', ' ', 'no', ' ', 'g']


In [ ]:
result = [item for item in result if item.strip()]  # Remove empty strings
print(result)

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius--though', 'a', 'good', 'fellow', 'enough--so', 'it', 'was', 'no', 'g']


In [28]:
result = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)  # Split by punctuation and whitespace
# preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in result if item.strip()]
print(len(preprocessed))
print(preprocessed[:99])

4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself', 'in', 'a', 'villa', 'on', 'the', 'Riviera', '.', '(', 'Though', 'I', 'rather', 'thought', 'it', 'would', 'have', 'been', 'Rome', 'or', 'Florence', '.', ')', '"', 'The', 'height', 'of', 'his', 'glory', '"', '--', 'that', 'was', 'what', 'the', 'women', 'called', 'it', '.', 'I', 'can', 'hear', 'Mrs', '.', 'Gideon', 'Thwing', '--', 'his', 'last', 'Chicago', 'sitter']


In [29]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print("Vocabulary size:", vocab_size)

Vocabulary size: 1130


In [31]:
vocab = {token: integer for integer, token in enumerate(all_words)}
vocab['Jack']

57

In [32]:
 for i, item in enumerate(vocab.items()):
    print(item)
    if i > 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


In [40]:
class simpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {v: k for k, v in vocab.items()}

    def encoder(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[token] for token in preprocessed]
        return ids

    def decoder(self, tokens):
        text = " ".join([self.int_to_str[token] for token in tokens])
        text = re.sub(r'\s([,.:;?_!"()\']|--)\s', r'\1', text)  # Remove spaces around punctuation
        return text

In [41]:
tokenizer = simpleTokenizerV1(vocab)

# text = "Jack and Jill went up the hill."
text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
tokens = tokenizer.encoder(text)
print("Tokens:", tokens)

text_reconstructed = tokenizer.decoder(tokens)
print("Reconstructed text:", text_reconstructed)

Tokens: [1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
Reconstructed text: " It's the last he painted,you know," Mrs.Gisburn said with pardonable pride .


In [43]:
text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encoder(text)

KeyError: 'Hello'

In [64]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

In [65]:
vocab = {token: integer for integer, token in enumerate(all_tokens)}
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [66]:
class simpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {v: k for k, v in vocab.items()}

    def encoder(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int else "<|unk|>"
            for item in preprocessed
        ]
        ids = [self.str_to_int[token] for token in preprocessed]
        return ids

    def decoder(self, tokens):
        text = " ".join([self.int_to_str[token] for token in tokens])
        text = re.sub(r'\s([,.:;?_!"()\']|--)\s', r'\1', text)  # Remove spaces around punctuation
        return text

In [67]:
tokenizer = simpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [69]:
tokenizer.encoder(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [70]:
tokenizer.decoder(tokenizer.encoder(text))

'<|unk|>,do you like tea?<|endoftext|> In the sunlit terraces of the <|unk|> .'

In [77]:
import importlib
import tiktoken

print("ticktoken version:", importlib.metadata.version("tiktoken"))
tokenizer = tiktoken.get_encoding("gpt2")

# text = "Jack and Jill went up the hill."
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)
tokens = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print("Tokens:", tokens)

text_reconstructed = tokenizer.decode(tokens)
print("Reconstructed text:", text_reconstructed)

ticktoken version: 0.9.0
Tokens: [15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]
Reconstructed text: Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [80]:
print(raw_text[:100])

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))
context_length = 4

enc_sample = enc_text[:50:]
print(enc_sample)

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g
5145
[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138, 257, 7026, 15632, 438, 2016, 257, 922, 5891, 1576, 438, 568, 340, 373, 645, 1049, 5975, 284, 502, 284, 3285, 326, 11, 287, 262, 6001, 286, 465, 13476, 11, 339, 550, 5710, 465, 12036, 11, 6405, 257, 5527, 27075, 11]


In [86]:
import torch
from torch.utils.data import Dataset, DataLoader
class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Text is too short for the specified max_length."

        # Use a sliding window approch to create input-target pairs
        for i in range(0, len(token_ids) - max_length, stride):
            input_ids = token_ids[i:i + max_length]
            target_ids = token_ids[i + 1:i + max_length + 1]
            self.input_ids.append(torch.tensor(input_ids))
            self.target_ids.append(torch.tensor(target_ids))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

def create_dataloaderv1(text, batch_size=4, max_length=256, stride=128, shuffle=False, drop_last=False, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(text, tokenizer, max_length, stride)

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers,
        )
    return dataloader

In [88]:
dataloader = create_dataloaderv1(raw_text, batch_size = 1, max_length=4, stride=4, shuffle=False, drop_last=False, num_workers=0)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[1807, 3619,  402,  271]]), tensor([[ 3619,   402,   271, 10899]])]


In [89]:
dataloader = create_dataloaderv1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [90]:
input_ids = torch.tensor([2, 3, 5, 1])
vocab_size = 6
output_dim = 3

torch.manual_seed(42)
embdedding = torch.nn.Embedding(vocab_size, output_dim)
embedded = embdedding(input_ids)
print(embedded)

tensor([[ 0.8008,  1.6806,  0.3559],
        [-0.6866,  0.6105,  1.3347],
        [ 0.8599, -0.3097, -0.3957],
        [ 0.4396, -0.7581,  1.0783]], grad_fn=<EmbeddingBackward0>)


In [93]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(inputs)

token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
torch.Size([8, 4, 256])


In [96]:
context_length = 4
position_embedding_layer = torch.nn.Embedding(context_length, output_dim)

pos_embeddings = position_embedding_layer(torch.arange(context_length))
print(pos_embeddings)
print(pos_embeddings.shape)

input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

tensor([[-0.1185,  0.5712, -0.6269,  ...,  0.1241, -0.5065,  0.1380],
        [-0.7440,  0.6457, -0.3528,  ...,  0.6297,  0.5204, -0.3531],
        [-0.4593,  0.1106,  0.5295,  ...,  0.0799,  0.9920, -0.3130],
        [-1.6532, -0.6109,  1.7626,  ...,  0.0118,  0.1272,  1.0433]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([4, 256])
torch.Size([8, 4, 256])
